# Interactive map of in-person attendance in Texas public schools

## Load required python libraries

In [2]:
import ipystata
import pandas as pd
import geopandas as gpd
import folium
import sys
from folium import plugins
import branca.colormap as cm
#import plotly.express as px
import numpy as np

IPyStata is loaded in batch mode.


## Read-in most recent Stata .dta file, subset, create labels for the Folum map, and output Pandas dataframe

In [12]:
%%stata -o geocovid_df -np
use "/Volumes/Shared/data/goofin/texas_covid/final/district_tea_covid_data_27sep2020_13apr2021.dta", clear
fre submission_counter
keep if submission_counter==19 & M_MISSING_GEOID==3
keep FID d_*per10* d_petblachispc_pct_2020 district distname d_petallc_2020 d_enrollment_change_pct ///
    d_enrollment_change d_stu_oncampus_pct d_stu_enroll d_stu_oncampus d_total_stu d_total_staff ///
    d_staff_cases_pct d_stu_cases_pct lea_locale date_range
drop if FID==.
local xform d_petallc d_stu_enroll d_stu_oncampus d_total_stu d_total_staff d_staff_cases_pct d_stu_cases_pct
quietly foreach var of varlist `xform' {
    if substr("`var'", -3, .)=="pct" {
        replace `var'=round(`var', .1)
        tostring `var', replace usedisplay force
        replace `var'="Suppressed or not reported" if `var'=="." | `var'==".a" | `var'==".b" | `var'==""        
        replace `var'=`var'+"%" if `var'!="NA"
    }
    else {
        format %10.0fc `var'
        tostring `var', replace usedisplay force
        replace `var'="NA" if `var'=="." | `var'==".a" | `var'==".b" | `var'==""                
    }
}

gen d_case_rate_cases=d_stu_cases_pct+" ("+d_total_stu+")"
replace d_case_rate_cases="Suppressed or not reported" if d_total_stu=="."

*Preparing variables for map and tooltips
local string d_stu_oncampus_pct d_enrollment_change_pct d_staff_cases_per100 d_stu_cases_per1000
foreach var of varlist `string' {
    replace `var'=round(`var', .1)
    format `var' %5.1f
    tostring `var', force usedisplay gen(LABEL_`var')
    if substr("`var'", -3, .)=="pct" {
        replace LABEL_`var'=LABEL_`var'+"%"
    }
     replace LABEL_`var'="Suppressed or not reported" if ///
            LABEL_`var'=="." | LABEL_`var'==".a" | LABEL_`var'==".b" | LABEL_`var'==""      
}

sdecode lea_local, replace
gen DISTRICT_N=district
mdesc DISTRICT_N
di "Done"

## Linking the pandas dataframe to the 2018/19 Texas district .geojson file. 
District boundary data are available through TEA's API which is embedded on this page: https://schoolsdata2-tea-texas.opendata.arcgis.com/datasets/e115fed14c0f4ca5b942dc3323626b1c_0

In [13]:
##Using the geopandas python library to convert the shapefile to a pandas dataframe.
district_geo = gpd.read_file('/Volumes/Shared/data/goofin/texas_covid/geodata/district/Districts_2018-2019_SIMPLIFIED.json')
district_geo=district_geo[['NAME2','geometry','DISTRICT_N']]
district_geo=district_geo.dropna(how='any', subset=['geometry'])
##Next, let's create a consolidated dataset
combined=district_geo.merge(geocovid_df,on="DISTRICT_N")
combined=combined[['DISTRICT_N', 'district', 'NAME2','geometry', 'd_stu_cases_per1000','d_staff_cases_per100','LABEL_d_staff_cases_per100','LABEL_d_enrollment_change_pct','d_enrollment_change_pct','d_case_rate_cases', 'd_stu_oncampus_pct','LABEL_d_stu_oncampus_pct','d_stu_enroll_2021', 'date_range','lea_locale','LABEL_d_stu_cases_per1000']]
combined=combined.dropna(how='any', subset=['geometry'])
#Creating separate data frames for each outcome
enrollch=combined[['DISTRICT_N', 'district', 'NAME2','geometry','d_case_rate_cases', 'd_enrollment_change_pct','LABEL_d_enrollment_change_pct','d_stu_enroll_2021', 'date_range','lea_locale']]
oncampus=combined[['DISTRICT_N', 'district', 'NAME2','geometry','d_case_rate_cases', 'd_stu_oncampus_pct','LABEL_d_stu_oncampus_pct','d_stu_enroll_2021', 'date_range','lea_locale']]
staff=combined[['DISTRICT_N', 'district', 'NAME2','geometry','d_staff_cases_per100','LABEL_d_staff_cases_per100','d_case_rate_cases', 'd_stu_enroll_2021', 'date_range','lea_locale']]
student=combined[['DISTRICT_N', 'district', 'NAME2','geometry','d_stu_cases_per1000','LABEL_d_stu_cases_per1000','d_case_rate_cases', 'd_stu_enroll_2021', 'date_range','lea_locale']]

## Creating interactive Folium map of in-person attendance


In [14]:
#Defining the color scale
colormap = cm.LinearColormap(colors=['white','#3998A3'], index=[0,100],vmin=0,vmax=100)
colormap

##creating on-campus instruction rates
#Defining the color scale
colormap.caption = 'Percentage of students receiving on-campus instruction (January 2021)'
##Now,district_geo let's just create an empty map zoomed in on the centroid of Texas.
api='sk.eyJ1IjoibWFyc2hhbGx3ZyIsImEiOiJja2pxM2JqNHUwa29tMnRwNXkybnJ5Z282In0.5ExddeMXlLn6L_3nRON88w'
tileurl = 'https://api.mapbox.com/styles/v1/mapbox/light-v10/tiles/{z}/{x}/{y}?access_token=' + str(api)
map=folium.Map(
    location=[31.60631045236211, -99.46453405204277], 
    zoom_start=5,
    #min_zoom=6,
    tiles=tileurl,   
    attr='Mapbox | Texas Public Schools COVID-19 Data | <a href="https://gibsonconsult.com/">Gibson Consulting Group</a>',
    control=False
)

style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['d_stu_oncampus_pct']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

NIL=folium.features.GeoJson(
    oncampus,
    style_function=style_function,   
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['NAME2','LABEL_d_stu_oncampus_pct','d_case_rate_cases','d_stu_enroll_2021','lea_locale','date_range'],
        aliases=['District: ','Percentage of students on campus:','Cumulative case rate (cumulative cases): ','Total student enrollment: ','Locale: ','Reporting period: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
        sticky=True
    )
)

colormap.add_to(map)
map.add_child(NIL)

distsearch = plugins.Search(
    layer=NIL,
    geom_type="Polygon",
    placeholder="Search for a district",
    collapsed=False,
    search_label="NAME2",
    weight=3,
).add_to(map)

map.save('/Volumes/Shared/data/goofin/texas_covid/results/covid_map_on_campus.html')
map